## Delhi House Price Predition

In [1]:
import pandas as pd
import numpy as np

### Load the data

In [3]:
data = pd.read_csv("Delhi_v2.csv")

### Data info

In [127]:
data.head()


,price,area,latitude,longitude,Bedrooms,Bathrooms,Status,neworold,Price_sqft,type_of_building_Individual House
0,5600000.0,1350.0,28.608850,77.460560,3.0,3.0,1,0,4148.148148,0
1,8800000.0,1490.0,28.374236,76.952416,3.0,3.0,0,0,5906.040268,0
2,16500000.0,2385.0,28.645769,77.385110,4.0,5.0,0,0,6918.238994,0
3,3810000.0,1050.0,28.566914,77.436434,2.0,2.0,0,0,3628.571429,0
4,6200000.0,1350.0,28.520732,77.356491,2.0,2.0,0,1,4592.592593,0


In [123]:
data.shape

(7689, 10)

In [125]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7689 entries, 0 to 7737
Data columns (total 10 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   price                              7689 non-null   float64
 1   area                               7689 non-null   float64
 2   latitude                           7689 non-null   float64
 3   longitude                          7689 non-null   float64
 4   Bedrooms                           7689 non-null   float64
 5   Bathrooms                          7689 non-null   float64
 6   Status                             7689 non-null   int64  
 7   neworold                           7689 non-null   int64  
 8   Price_sqft                         7689 non-null   float64
 9   type_of_building_Individual House  7689 non-null   int64  
dtypes: float64(7), int64(3)
memory usage: 660.8 KB


### Chekcing for missing values

In [37]:
print(data.isnull().sum())

price               0
Address             0
area                0
latitude            0
longitude           0
Bedrooms            0
Bathrooms           0
Status              0
neworold            0
type_of_building    0
Price_sqft          0
dtype: int64


### Dropping irrelevant columns

In [15]:
data.drop(["Unnamed: 0", "desc", "Lift", "Landmarks", "parking", "Furnished_status", "Balcony"], axis=1, inplace=True)


In [19]:
data.shape

(7738, 11)

### Handling Missing values in one column only -> Status

In [27]:
print(data["Status"].unique())



['Under Construction' 'Ready to Move' nan]


In [29]:
print(data["Status"].value_counts())


Status
Ready to Move         7148
Under Construction      16
Name: count, dtype: int64


In [35]:
data["Status"].fillna("Ready to Move", inplace=True)


### Describing the data now

In [135]:
data.describe()

,price,area,latitude,longitude,Bedrooms,Bathrooms,Status,neworold,Price_sqft,type_of_building_Individual House
count,7.689000e+03,7689.000000,7689.000000,7689.000000,7689.000000,7689.000000,7689.000000,7689.000000,7689.000000,7689.000000
mean,8.339062e+06,1411.493432,28.551939,77.273031,2.711146,2.503577,0.002081,0.891663,5548.857978,0.196645
std,7.234203e+06,720.126904,0.107413,0.180645,0.878363,0.868886,0.045572,0.310825,2412.071289,0.397487
min,1.700000e+06,501.000000,28.240023,76.884101,2.000000,2.000000,0.000000,0.000000,2100.000000,0.000000
25%,4.200000e+06,990.000000,28.455376,77.078564,2.000000,2.000000,0.000000,1.000000,3952.569170,0.000000
50%,6.000000e+06,1250.000000,28.574637,77.344698,3.000000,2.000000,0.000000,1.000000,4978.877489,0.000000
75%,9.500000e+06,1650.000000,28.642231,77.421054,3.000000,3.000000,0.000000,1.000000,6363.636364,0.000000
max,8.500000e+07,9500.000000,28.799748,77.688028,10.000000,10.000000,1.000000,1.000000,44378.698225,1.000000


In [60]:
print(data.duplicated().sum())  # Should be 0 ideally


0


In [58]:
data.drop_duplicates(inplace=True)


In [76]:
print(data.dtypes)


price                                float64
area                                 float64
latitude                             float64
longitude                            float64
Bedrooms                             float64
Bathrooms                            float64
Status                                 int64
neworold                               int64
Price_sqft                           float64
type_of_building_Individual House       bool
dtype: object


In [64]:
data.drop(columns=["Address"], inplace=True)


### Numerical Encoding (Label Encoding)
#### column status and neworold-> converting them to numerical data

In [68]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

data["Status"] = le.fit_transform(data["Status"])  # Converts "Ready to Move" → 0, "Under Construction" → 1
data["neworold"] = le.fit_transform(data["neworold"])  # Converts categories to 0/1


### converting type_of_building column 
#### Since type_of_building has multiple unique categories
#### so we cannot use Label Encoding because it would treat "Villa" > "Apartment", which makes no sense.

### 👉 Instead, we use One-Hot Encoding:

In [74]:
data = pd.get_dummies(data, columns=["type_of_building"], drop_first=True)


In [121]:
print(data.dtypes)


price                                float64
area                                 float64
latitude                             float64
longitude                            float64
Bedrooms                             float64
Bathrooms                            float64
Status                                 int64
neworold                               int64
Price_sqft                           float64
type_of_building_Individual House      int64
dtype: object


In [89]:
data["type_of_building_Individual House"] = data["type_of_building_Individual House"].astype(int)


## Split the data into training set and test set

In [94]:
from sklearn.model_selection import train_test_split

# Define features (X) and target (y)
X = data.drop(columns=["price"])  # Drop 'price' since it's what we want to predict
y = data["price"]  # Target variable

# Split into training (80%) and testing (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Training samples: {len(X_train)}, Testing samples: {len(X_test)}")


Training samples: 6151, Testing samples: 1538


## Train the Machine Learning Model
### Start with a simple model like Linear Regression:

In [98]:
from sklearn.linear_model import LinearRegression

# Initialize model
model = LinearRegression()

# Train the model
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)


In [100]:
y_pred

array([4081097.55774014, 5502341.31381713,  581193.53965276, ...,
       2230370.99095581, 5077401.78426568, 8414763.73177579])

In [102]:
from sklearn.metrics import mean_absolute_error, r2_score

# Evaluate performance
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Absolute Error: {mae:.2f}")
print(f"R² Score: {r2:.2f}")



Mean Absolute Error: 1025821.40
R² Score: 0.93


#### 🔥 My R² Score = 0.93, which means my model explains 93% of the variance in house prices.

#### However, Mean Absolute Error (MAE) = ₹10,25,821, which means, on average, my predictions are off by ~10 lakh rupees. That could be improved. 

### Feature Scaling (for models like Linear Regression)

In [107]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [111]:
# Train the model again with scaled data
model.fit(X_train_scaled, y_train)

# Make predictions
y_pred_scaled = model.predict(X_test_scaled)

# Evaluate performance
from sklearn.metrics import mean_absolute_error, r2_score

mae_scaled = mean_absolute_error(y_test, y_pred_scaled)
r2_scaled = r2_score(y_test, y_pred_scaled)

print(f"Mean Absolute Error (After Scaling): {mae_scaled:.2f}")
print(f"R² Score (After Scaling): {r2_scaled:.2f}")


Mean Absolute Error (After Scaling): 1025821.40
R² Score (After Scaling): 0.93


# 🔹 Save the Trained Model

In [114]:
import joblib

# Save the trained model
joblib.dump(model, "house_price_model.pkl")

# Save the scaler too (if used)
joblib.dump(scaler, "scaler.pkl")


['scaler.pkl']

# 🔹 Create a Python API (Flask or FastAPI)

In [117]:
pip install flask joblib pandas numpy


Note: you may need to restart the kernel to use updated packages.


In [137]:
X = data[["area", "latitude", "longitude", "Bedrooms", "Bathrooms", "Status", "neworold", "Price_sqft", "type_of_building_Individual House"]]
y = data["price"]


In [143]:
X

,area,latitude,longitude,Bedrooms,Bathrooms,Status,neworold,Price_sqft,type_of_building_Individual House
0,1350.0,28.608850,77.460560,3.0,3.0,1,0,4148.148148,0
1,1490.0,28.374236,76.952416,3.0,3.0,0,0,5906.040268,0
2,2385.0,28.645769,77.385110,4.0,5.0,0,0,6918.238994,0
3,1050.0,28.566914,77.436434,2.0,2.0,0,0,3628.571429,0
4,1350.0,28.520732,77.356491,2.0,2.0,0,1,4592.592593,0
...,...,...,...,...,...,...,...,...,...
7733,1095.0,28.635272,77.370395,2.0,2.0,0,1,7214.611872,0
7734,1060.0,28.581431,77.452819,2.0,2.0,0,1,4254.716981,0
7735,1898.0,28.625850,77.435336,4.0,3.0,0,1,3688.092729,0
7736,1400.0,28.701622,77.430153,3.0,3.0,0,1,4642.857143,0
